In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

District names and district unique identifiers

In [ ]:
districtnames = []
districtlink = []

for i in range (65,90):

    url = 'https://data.nysed.gov/lists.php?start={}&type=district'
    url = url.format(i)
    response = requests.get(url)

    rawhtml = response.text
    
    soup = BeautifulSoup(rawhtml,'html.parser')
    
    districtnamesraw = soup.find_all('div', attrs = {'class' : 'title'})

    for i in districtnamesraw:
        name = i.find('a')
        if name == None:
            pass
        else:
            district = name.text
            districtnames.append(district)
            link = name.get('href')
            link = link[-12:]
            districtlink.append(link)

cleaneddistrictnames = []

for i in districtnames:
    newnames = i.rstrip()
    cleaneddistrictnames.append(newnames)

Graduation and Dropout Rate

In [ ]:
graduationratelist = []
dropoutratelist = []

for i in districtlink:
    
    url = 'https://data.nysed.gov/gradrate.php?year=2020&instid={}'
    
    url = url.format(i)
    
    response = requests.get(url)

    rawhtml = response.text

    soup = BeautifulSoup(rawhtml,'html.parser')

    rawgraduationdata = soup.find_all('tr')

    allstudentsrow = rawgraduationdata[2]

    graduationrate = allstudentsrow.find('td', attrs = {'data-label' : 'Percentage of Graduates'})

    graduationrate = graduationrate.text
    
    dropoutrate = allstudentsrow.find('td', attrs = {'data-label' : 'Percentage of Dropout'})
    
    dropoutrate = dropoutrate.text

    graduationratelist.append(graduationrate)
    
    dropoutratelist.append(dropoutrate)
    
    print('graduation rate was ' + str(graduationrate) + ' and the drop out rate was ' + dropoutrate)

In [ ]:
# remove % in each index

graduationratelist = [x.replace('%', '') for x in graduationratelist]
dropoutratelist = [x.replace('%', '') for x in dropoutratelist]

Funding Per Student

In [ ]:
fundingperstudentlist = []

for i in districtlink:
    
    url = 'https://data.nysed.gov/expenditures.php?year=2020&instid={}'.format(i)

    response = requests.get(url)

    rawhtml = response.text

    soup = BeautifulSoup(rawhtml,'html.parser')

    fundingperstudent = soup.find_all('div', attrs = {'class' : 'bullet-item columns'})

    if len(fundingperstudent) <1:

        fundingperstudentlist.append('N/A')

        print("No funding available")

    else:

        fundingperstudent = fundingperstudent[5].find('h4').text

        print(fundingperstudent)

        fundingperstudentlist.append(fundingperstudent)

In [ ]:
#Remove $ and commas in each index

fundingperstudentlist = [x.replace('$', '') for x in fundingperstudentlist]
fundingperstudentlist = [x.replace(',', '') for x in fundingperstudentlist]

Total Amount of Students Enrolled Per District

In [ ]:
totalenrollmentlist = []

for i in districtlink:

    url = 'https://data.nysed.gov/enrollment.php?year=2020&instid={}'.format(i)

    response = requests.get(url)

    rawhtml = response.text

    soup = BeautifulSoup(rawhtml,'html.parser')

    totalenrollment = soup.find('div', attrs = {'class' : 'highlight blue spacing-top-15'})

    totalenrollment = totalenrollment.text

    totalenrollment = totalenrollment[17:]
    
    totalenrollmentlist.append(totalenrollment)

    print(totalenrollment)

In [ ]:
#Strip off commas
totalenrollmentlist = [x.replace(',', '') for x in totalenrollmentlist]
print(totalenrollmentlist)

Amount of Students that were Male

In [ ]:
enrollmentlistbymale = []

for i in districtlink:

    url = 'https://data.nysed.gov/enrollment.php?year=2020&instid={}'.format(i)
    
    response = requests.get(url)

    rawhtml = response.text

    soup = BeautifulSoup(rawhtml,'html.parser')

    maleenrollment = soup.find_all('div', attrs = {'class' : 'small-12 large-6 medium-6 bullet-item columns'})

    maleenrollment = maleenrollment[0]

    maleenrollment = maleenrollment.find('h4')
    
    maleenrollment = maleenrollment.text

    print(maleenrollment)
    
    enrollmentlistbymale.append(maleenrollment)


In [ ]:
#Strip off commas
enrollmentlistbymale = [x.replace(',', '') for x in enrollmentlistbymale]

Amount of Students that were Female

In [ ]:
enrollmentlistbyfemale = []


for i in districtlink:

    url = 'https://data.nysed.gov/enrollment.php?year=2020&instid={}'.format(i)

    response = requests.get(url)

    rawhtml = response.text

    soup = BeautifulSoup(rawhtml,'html.parser')

    femaleenrollment = soup.find_all('div', attrs = {'class' : 'small-12 large-6 medium-6 bullet-item columns'})

    femaleenrollment = femaleenrollment[2]

    femaleenrollment = femaleenrollment.find('h4')

    femaleenrollment = femaleenrollment.text

    print(femaleenrollment)

    enrollmentlistbyfemale.append(femaleenrollment)

In [ ]:
#Strip off commas 

enrollmentlistbyfemale = [x.replace(',', '') for x in enrollmentlistbyfemale]
print(enrollmentlistbyfemale)

Percentage of Minority Students in District

In [ ]:
whitestudentsenrolled = []

for i in districtlink:


    url = 'https://data.nysed.gov/enrollment.php?year=2020&instid={}'.format(i)

    response = requests.get(url)

    rawhtml = response.text

    soup = BeautifulSoup(rawhtml,'html.parser')

    amount = soup.find(text = 'WHITE')

    if amount == None:
        print('no info available')
        whitestudentsenrolled.append('N/A')
    else:
        amount = amount.find_next().text
        print(amount)
        whitestudentsenrolled.append(amount)

In [ ]:
whitestudentsenrolled = [x.replace('\n', '') for x in whitestudentsenrolled]
whitestudentsenrolled = [x.replace(',', '') for x in whitestudentsenrolled]
print(whitestudentsenrolled)

In [ ]:
finaldf = pd.DataFrame(list(zip(cleaneddistrictnames,graduationratelist,dropoutratelist,fundingperstudentlist,totalenrollmentlist,enrollmentlistbymale,enrollmentlistbyfemale,whitestudentsenrolled)), columns = ['DistrictName','GraduationPercentageRate','DropoutPercentageRate','FundingPerStudent','TotalEnrolled','MalesEnrolled','FemalesEnrolled','WhiteStudentsEnrolled'])

In [ ]:
finaldf.columns

In [ ]:
#Remove all rows that are N/A in column in order to transform into % of ethnicity column
finaldf = finaldf[finaldf.WhiteStudentsEnrolled != 'N/A']
finaldf.head(5)

In [ ]:
#create column that represents % of students that were minority

finaldf['Percentage of Minority Students'] = ((pd.to_numeric(finaldf['TotalEnrolled']) - pd.to_numeric(finaldf['WhiteStudentsEnrolled'])) * 100) /  pd.to_numeric(finaldf['TotalEnrolled'])
finaldf['Percentage of Minority Students'] = finaldf['Percentage of Minority Students'].round(1)
finaldf.head()

In [ ]:
finaldf.to_csv('../Datasets/PreliminaryDistrictData1.csv')

In [8]:
import pandas as pd
finaldf = pd.read_csv('../Datasets/PreliminaryDistrictData1.csv')

Add Median Income per District to DataFrame

In [2]:
medianincome = pd.read_csv('../Datasets/medianincome.csv')
print(medianincome.head())

                         DistrictName  HouseholdIncome
0     ADDISON CENTRAL SCHOOL DISTRICT     59328.000000
1  ADIRONDACK CENTRAL SCHOOL DISTRICT     63986.357143
2       AFTON CENTRAL SCHOOL DISTRICT     54818.285714
3       AKRON CENTRAL SCHOOL DISTRICT     76903.750000
4         ALBANY CITY SCHOOL DISTRICT     61049.181818


In [3]:
#Clean district names to match the original district names

medianincome = medianincome.replace('CENTRAL SCHOOL DISTRICT','CSD', regex = True)
medianincome = medianincome.replace('UNION FREE SCHOOL DISTRICT','UFSD', regex = True)

Merge Dataframes

In [4]:
mergeddf = pd.merge(finaldf, medianincome, on = 'DistrictName', how = 'left')
mergeddf = mergeddf.drop('WhiteStudentsEnrolled', axis = 1)
mergeddf['Percentage of Minority Students'] = mergeddf['Percentage of Minority Students'].round(1)
mergeddf['HouseholdIncome'] = mergeddf['HouseholdIncome'].round(1)

mergeddf.head()

,Unnamed: 0,DistrictName,GraduationPercentageRate,DropoutPercentageRate,FundingPerStudent,TotalEnrolled,MalesEnrolled,FemalesEnrolled,Percentage of Minority Students,HouseholdIncome
0,0,ADDISON CSD,90,3,23833.02,976,503,473,3.2,59328.0
1,1,ADIRONDACK CSD,92,5,14013.61,1145,575,570,1.7,63986.4
2,2,AFTON CSD,83,10,24128.02,507,262,245,8.9,54818.3
3,3,AKRON CSD,95,3,17552.56,1334,667,667,17.5,76903.8
4,4,ALBANY CITY SCHOOL DISTRICT,71,12,22382.65,8610,4444,4166,80.2,61049.2


In [5]:
mergeddf.to_csv('../Datasets/PreliminaryDistrictDataWITHMEDIANINCOME2.csv')

In [9]:
# sql dump
finaldf = finaldf.drop('Unnamed: 0',axis=1)
finaldf['GraduationPercentageRate'] = pd.to_numeric(finaldf['GraduationPercentageRate'], errors='coerce')
finaldf['DropoutPercentageRate'] = pd.to_numeric(finaldf['DropoutPercentageRate'], errors='coerce')
finaldf = finaldf.replace('CITY SCHOOL DISTRICT','CITY SD', regex = True)
finaldf = finaldf.replace('COMMON SCHOOL DISTRICT','COMN SD', regex = True)
finaldf = finaldf.replace('CENTRAL HS DISTRICT','CENTRAL SCHOOL DISTRICT',regex = True)

In [10]:
finaldf.to_csv('../SQL/DistrictData.csv',index=False)